# Import Necessary Headers

In [1]:
import pandas as pd
import numpy as np
import shap
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load Updated CSV into Pandas Dataframe¶

In [2]:
# Hong et al. Dataset
df = pd.read_csv("hong_et_al/df_updt.csv")

In [3]:
# Drop unnecessary columns
df = df.drop(columns=df.filter(like='chrkidneydisease').columns)
df = df.drop(columns=df.filter(like='chrkidneydisease').columns)
df = df.drop(columns=df.filter(like='(pcp)screen').columns)
df = df.drop(columns=df.filter(regex='^cc_').columns)
df = df.dropna(subset=['egfr_CKD_EPI'])

# Impute median data for any remaining NaN values in numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Impute mode data for any remaining NaN values in categorical columns
categorical_cols = ['dep_name', 'gender', 'ethnicity', 'race', 'lang', 'religion',
                    'maritalstatus', 'employstatus', 'insurance_status', 'disposition', 'arrivalmode',
                    'arrivalmonth', 'arrivalday', 'arrivalhour_bin', 'previousdispo']

# One-hot encode categorical features
df = pd.get_dummies(df, columns=categorical_cols)

# Ensure no infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.median(), inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212834 entries, 6 to 560485
Columns: 853 entries, esi to previousdispo_Transfer to Another Facility
dtypes: bool(100), float64(386), int64(367)
memory usage: 1.2 GB


# Define Train and Test Data Splits

In [5]:
# Exclude target variable from features frame
X = df.drop(columns=df.filter(like='egfr').columns)
y = df['egfr_CKD_EPI']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Ensure no NaN values in tensors
assert not X_train_tensor.isnan().any(), "NaN values found in X_train_tensor"
assert not y_train_tensor.isnan().any(), "NaN values found in y_train_tensor"

# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Neural Network Architecture

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

input_dim = X_train.shape[1]
model = SimpleNN(input_dim)

# Training Loop

In [7]:
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 40
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(test_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch 1/40, Training Loss: 389.2960932293318, Validation Loss: 88.32814370834076
Epoch 2/40, Training Loss: 153.0666476295927, Validation Loss: 51.53857711127571
Epoch 3/40, Training Loss: 121.67321357884742, Validation Loss: 53.80292921667701
Epoch 4/40, Training Loss: 103.15638757280638, Validation Loss: 63.6793866429601
Epoch 5/40, Training Loss: 93.02854951545288, Validation Loss: 35.831682799456715
Epoch 6/40, Training Loss: 90.17842726365197, Validation Loss: 44.31403050408349
Epoch 7/40, Training Loss: 83.563485208287, Validation Loss: 32.23780753018262
Epoch 8/40, Training Loss: 79.82635317131883, Validation Loss: 28.8643836030015
Epoch 9/40, Training Loss: 76.62734095972313, Validation Loss: 28.92677808452297
Epoch 10/40, Training Loss: 74.99856930216288, Validation Loss: 25.661893273259068
Epoch 11/40, Training Loss: 72.52895813389678, Validation Loss: 24.40857630234223
Epoch 12/40, Training Loss: 69.33904329864032, Validation Loss: 24.395817803906965
Epoch 13/40, Training Lo

# Evaluate Accuracy on Test Set

In [8]:
# Switch the model to evaluation mode
model.eval()

# Initialize lists to store predictions and true values
all_predictions = []
all_labels = []

# Predict and evaluate
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        all_predictions.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Concatenate all batches into a single array
all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

# Calculate metrics
mse = mean_squared_error(all_labels, all_predictions)
mae = mean_absolute_error(all_labels, all_predictions)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 14.60035514831543
Mean Absolute Error: 2.9469079971313477
